<a href="https://colab.research.google.com/github/sagrawal128/validatrix/blob/main/Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This documentation will take you through installing and using the validatrix library.

You can install directly from the github link as below:

In [ ]:
pip install git+https://github.com/sagrawal128/validatrix

## Usage

In this section, we would demonstrate how can you use validatrix to validate the data using rules. You need to create a rule object first and then call the validate method. Some of the rules are prebuilt into the library such as HasColumns and HasSchema. Adding more rules would be discussed in a later section.

In [2]:
from validatrix import HasColumns
import pandas as pd

In [3]:
data = pd.read_csv("./sample_data/california_housing_train.csv")

In [5]:
data.head

<bound method NDFrame.head of        longitude  latitude  ...  median_income  median_house_value
0        -114.31     34.19  ...         1.4936             66900.0
1        -114.47     34.40  ...         1.8200             80100.0
2        -114.56     33.69  ...         1.6509             85700.0
3        -114.57     33.64  ...         3.1917             73400.0
4        -114.57     33.57  ...         1.9250             65500.0
...          ...       ...  ...            ...                 ...
16995    -124.26     40.58  ...         2.3571            111400.0
16996    -124.27     40.69  ...         2.5179             79000.0
16997    -124.30     41.84  ...         3.0313            103600.0
16998    -124.30     41.80  ...         1.9797             85800.0
16999    -124.35     40.54  ...         3.0147             94600.0

[17000 rows x 9 columns]>

Let's define a rule that the dataframe contains longitude and latitude columns.

In [13]:
rule = HasColumns(columns=["longitude", "latitude"], allow_subset=True)
assert rule.validate(data)

Now let's try what would happen if we do not allow subsets.

In [11]:
rule = HasColumns(columns=["longitude", "latitude"], allow_subset=False)
rule.validate(data)

ValidationError: ignored

The above rule failed because we set the `allow_subset` argument false. This means that we want only and all the specified columns to be there. If such is not the case, we can resort to the default value of `allow_subset` to allow for a subset of column names.

Also note in the above example how the rule generated descriptive information about what were the unexpected columns it found.

In [15]:
rule = HasColumns(columns=["longitude", "latitude"], allow_subset=False)
result = rule.validate(data, silent=True)
print(result)

Validation Failed
Errors:
* Unexpected columns: {'median_house_value', 'housing_median_age', 'median_income', 'households', 'population', 'total_bedrooms', 'total_rooms'}


In the above example we see that an exception was not raised. Instead we get a nicer logging message. By default, when a rule fails an error is raised, this can be silenced using the `silent` argument in the Rule object.

### Rule as a function decorator

In this section we would demonstrated how the same rules could be incorporated in your python code directly to validate a function follows a given property.

This is quite useful in machine learning could where data might be changing properties based on external sources, or model might give spurious results. This approach helps separate the validation code from the business logic, as well as make your code more readable and debugging easier.

In [18]:
@HasColumns(columns=["longitude", "latitude"], allow_subset=False, raise_exc=False)
def load_data():
  return pd.read_csv("./sample_data/california_housing_train.csv")

df = load_data()

Output of function `load_data` failed to validate.
Validation Failed
Errors:
* Unexpected columns: {'median_house_value', 'housing_median_age', 'median_income', 'households', 'population', 'total_bedrooms', 'total_rooms'}


Each time load_data is called it is checked if the return values has the specified columns or not. Setting `raise_exc` to True would create an error each time the rule fails on load_data function. For this example, we set it to False.

We can also check a part of the output, instead of all the returned arguments.

In [19]:
@HasColumns(columns=["longitude", "latitude"], allow_subset=False, raise_exc=False, outputs="data")
def load_data():
  return {"data": pd.read_csv("./sample_data/california_housing_train.csv"), "message": "Requested data"}

df = load_data()

Output `data` of function `load_data` failed to validate.
Validation Failed
Errors:
* Unexpected columns: {'median_house_value', 'housing_median_age', 'median_income', 'households', 'population', 'total_bedrooms', 'total_rooms'}


Similarly, we can validate rules on inputs to the function as well.

In [20]:
@HasColumns(columns=["longitude", "latitude"], allow_subset=False, raise_exc=False, inputs="data")
def load_data(data, message):
  return {"data": data, "message": message}

data = pd.read_csv("./sample_data/california_housing_train.csv")
message = "Housing data"
df = load_data(data, message)

Input `data` of function `load_data` failed to validate.
Validation Failed
Errors:
* Unexpected columns: {'median_house_value', 'housing_median_age', 'median_income', 'households', 'population', 'total_bedrooms', 'total_rooms'}


# Extensibility

Often times you would like to bundle different rules together. For example, when encoding sensor data expectations, or a group of assumptions about a data frame. The class nature of the Rules exactly lets you manage your rules in one place for this purpose. Example you may create a new rule for sensor data as follow:

In [ ]:
from validatrix import Rule
class SensorRules(Rule):
  def __init__(columns, schema, range):
    ###
  def _validate(data):
    has_range()
    has_columns()
  def has_range():
    ###
  def has_columns():
    ###